In [79]:
%pip install pandas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pathlib 
import os
cur_path  = os.getcwd()

data_df = pd.read_csv(cur_path + '/account_data_info.csv')
history_df = pd.read_csv(cur_path + '/history.csv')
pd.set_option('display.min_rows', 150)





Note: you may need to restart the kernel to use updated packages.


/var/folders/sv/b643fd993ml6d9g_v9klf5mh0000gn/T/ipykernel_15419/138701694.py:14: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  history_df = pd.read_csv(cur_path + '/history.csv')


In [80]:
# deleting rows of account_data_info that dont appear in columns of history
filtered_data_df = data_df[data_df['Columns'].isin(history_df.columns)]

In [81]:
print(data_df.shape)
print(filtered_data_df.shape)
print(history_df.shape)

(111, 3)
(107, 3)
(673339, 106)


In [82]:
# show which rows were deleted
deleted_columns = data_df[~data_df['Columns'].isin(history_df.columns)]['Columns'].tolist()

print(deleted_columns)

['credit_code', 'closing_date', 'closing_date', 'closure_reason']


In [83]:
# determine columns with more than 95 percent NaN value
threshold_percent = 90

threshold_count = (threshold_percent / 100) * len(history_df)

missing_values_mask = history_df.isna()

missing_values_count = missing_values_mask.sum()

columns_with_more_than_x_missing = missing_values_count[missing_values_count > threshold_count].index.tolist()

print(columns_with_more_than_x_missing)
print(len(columns_with_more_than_x_missing))

['country_code', 'cashflows_custody_fee', 'arp_pension_origin', 'sss_location', 'sss_type', 'sss_agent', 'esir_number', 'shareholder_language', 'special_tag', 'conjunction', 'function_code', 'tms_settlement_location', 'loan_limit_override', 'special_fee_code', 'resp_specimen_plan']
15


In [84]:
# checking whether column with >90% NaN important or not
"""
NaN_column = 'resp_specimen_plan'   #replace with >90% Column

df_with_nan_values = history_df[history_df[NaN_column].isna()]

percentage_churn = (df_with_nan_values['label'] == 'Churn').mean() * 100
percentage_no_churn = (df_with_nan_values['label'] == 'No Churn').mean() * 100

print("CHURN PERCENTAGE", percentage_churn)
print("NO CHURN PERCENTAGE", percentage_no_churn)

"""

## trying to create table for all

label_col = 'label'

for i in columns_with_more_than_x_missing:
    df_with_nan_values = history_df[history_df[i].isna()]
    percentage_churn = (df_with_nan_values[label_col] == 'Churn').mean() * 100
    percentage_no_churn = (df_with_nan_values[label_col] == 'No Churn').mean() * 100

    print('Column:', i)
    print("Percentage Churn:", percentage_churn)
    print("Percentage No Churn:", percentage_no_churn)
    print(" ")





Column: country_code
Percentage Churn: 79.84261265974143
Percentage No Churn: 20.157387340258577
 
Column: cashflows_custody_fee
Percentage Churn: 79.96625770971234
Percentage No Churn: 20.033742290287655
 
Column: arp_pension_origin
Percentage Churn: 81.2465130981074
Percentage No Churn: 18.753486901892607
 
Column: sss_location
Percentage Churn: 79.8139341392097
Percentage No Churn: 20.186065860790293
 
Column: sss_type
Percentage Churn: 79.8139341392097
Percentage No Churn: 20.186065860790293
 
Column: sss_agent
Percentage Churn: 79.81438723395978
Percentage No Churn: 20.185612766040226
 
Column: esir_number
Percentage Churn: 79.96590066936274
Percentage No Churn: 20.034099330637268
 
Column: shareholder_language
Percentage Churn: 80.58965221937474
Percentage No Churn: 19.410347780625255
 
Column: special_tag
Percentage Churn: 79.16762375100494
Percentage No Churn: 20.83237624899506
 
Column: conjunction
Percentage Churn: 80.16084731361273
Percentage No Churn: 19.83915268638726
 
Co

In [85]:
# Total churn no churn percentage

percentage_churn = (history_df['label'] == 'Churn').mean() * 100
percentage_no_churn = (history_df['label'] == 'No Churn').mean() * 100
print("Total Chrun No Churn Percentage:")
print("Percentage Churn:", percentage_churn)
print("Percentage No Churn:", percentage_no_churn)



Total Chrun No Churn Percentage:
Percentage Churn: 79.96625770971234
Percentage No Churn: 20.033742290287655


In [86]:
# Create column bool: sss
    # SSS true if any of the sss are not NaN
    # else false

history_df['sss'] = history_df[['sss_location', 'sss_type', 'sss_agent']].notna().any(axis = 1)
